In [ ]:
# Tech Challenge - Fine-Tuning de Modelo (Llama 3 8B 4-bit via Unsloth)
#
# Este notebook implementa todo o fluxo solicitado:
#
# 1. Carregamento e limpeza do dataset (trn_teste.json -> filtrando linhas com content vazio).
# 2. Geração de arquivo limpo: data_titles_contents_cleaned.jsonl.
# 3. Conversão para formato de instrução: formatted_products_chat_data.json (instruction/input/output).
# 4. Preparação dos prompts no formato Alpaca-like.
# 5. Split treino / validação.
# 6. Baseline (inferência antes do fine-tuning).
# 7. Fine-tuning LoRA em 4-bit (Unsloth) do modelo unsloth/llama-3-8b-bnb-4bit.
# 8. Avaliação pós-treino (gera outputs para validação).
# 9. Métricas simples (ROUGE-1/2/L, BLEU opcional, overlap de tokens).
# 10. Salvamento de adaptadores e modelo fundido.
# 11. Função de busca de título (fuzzy) + geração de descrição (simulando pergunta do usuário).
# 12. Pipeline interativo (opcional).
# 13. Exportação de resultados / logs.
#
# Requisitos do desafio:
# - Pergunta do usuário sobre título de produto → recuperar título mais similar → gerar descrição aprendida.
# - Mostrar diferença antes e depois do fine-tune.
# - Documentar parâmetros principais.
#
# Observação: Para resultados melhores aumente:
# - num_train_epochs / max_steps
# - Tamanho do conjunto de treinamento
# - Qualidade da limpeza (remoção de duplicados / truncamento)

# Instalação de dependências principais
# Ajuste a linha do Unsloth se estiver em ambiente local sem GPU ou com CUDA diferente.
# %pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# %pip install -q transformers datasets accelerate peft bitsandbytes trl rapidfuzz evaluate sacrebleu
# %pip install -q bert-score

# Install necessary libraries
%pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
%pip install -q transformers datasets accelerate peft bitsandbytes trl rapidfuzz evaluate sacrebleu bert-score
%pip install -q bert-score rouge_score

import os
import json
import math
import random
import html
from pathlib import Path
from datetime import datetime
from collections import Counter

import torch
from datasets import load_dataset, Dataset, DatasetDict
from rapidfuzz import process, fuzz
from unsloth import FastLanguageModel, is_bfloat16_supported
from transformers import TrainingArguments
from trl import SFTTrainer
from evaluate import load as load_metric
from bert_score import score as bert_scorer

import pandas as pd
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Dispositivo detectado: {DEVICE}")

DATA_RAW_PATH = "trn_teste1.json"  # Ajuste se necessário (o arquivo bruto original)
CLEAN_JSONL_PATH = "data_titles_contents_cleaned.jsonl"
FORMAT_DATA_JSON = "formatted_products_chat_data.json"
RESULTS_DIR = "results"
os.makedirs(RESULTS_DIR, exist_ok=True)

MAX_SEQ_LENGTH = 2048
LOAD_IN_4BIT = True
DTYPE = None  # deixar None para Unsloth decidir


# Hiperparâmetros principais
EPOCHS = 3           # Aumentado para 3 épocas completas
LR = 2e-4
BATCH_SIZE = 2
GRAD_ACCUM = 4
WARMUP_STEPS = 10    # Aumentado ligeiramente para estabilizar o início
MAX_STEPS = -1       # Alterado para -1 para treinar por épocas
LOGGING_STEPS = 10

In [ ]:

print("Configurações carregadas.")

### 2. Limpeza e Preparação do Dataset
#
# Nesta seção, realizamos as seguintes etapas:
# 1.  **Limpeza do Arquivo Bruto**: Carregamos o `trn_teste1.json`, removemos registros com títulos ou conteúdos vazios/inválidos, eliminamos duplicatas e filtramos por comprimento para garantir a qualidade dos dados. O resultado é salvo em `data_titles_contents_cleaned.jsonl`.
# 2.  **Formatação para Instrução**: Convertemos os dados limpos para um formato de instrução (pergunta/resposta), ideal para o fine-tuning. O modelo aprenderá a associar um `input` (título do produto) a um `output` (descrição). O resultado é salvo em `formatted_products_chat_data.json`.
# 3.  **Criação do `Dataset` Hugging Face**: Carregamos os dados formatados em um objeto `Dataset` da biblioteca `datasets`, que é o padrão para treinamento no ecossistema Hugging Face.
# 4.  **Divisão em Treino e Validação**: Separamos o dataset em um conjunto de treino (95%) e um de validação (5%) para que possamos avaliar o desempenho do modelo em dados que ele não viu durante o treinamento.

# CÉLULA DE LIMPEZA DE DADOS (MELHORADA)
def limpar_arquivo_raw_para_jsonl(entrada: str, saida: str, min_content_len=20, max_content_len=4096):
    """
    Lê um arquivo JSONL, filtra, limpa e grava em um novo arquivo JSONL.
    - Remove linhas com 'title' ou 'content' vazios.
    - Decodifica entidades HTML (ex: &#8217; -> ').
    - Filtra conteúdos com base no comprimento (`min_content_len` e `max_content_len`).
    - Remove registros com 'title' duplicado, mantendo apenas a primeira ocorrência.
    """
    linhas_lidas, linhas_escritas = 0, 0
    titulos_vistos = set()  # Conjunto para rastrear títulos únicos

    with open(entrada, "r", encoding="utf-8") as fin, open(saida, "w", encoding="utf-8") as fout:
        for linha in fin:
            linhas_lidas += 1
            try:
                obj = json.loads(linha.strip())

                # Limpeza e decodificação
                title = html.unescape(str(obj.get("title", "")).strip())
                content = html.unescape(str(obj.get("content", "")).strip())

                # Validação: Título e conteúdo não vazios, conteúdo com tamanho mínimo e título não duplicado
                if title and content and min_content_len <= len(content) <= max_content_len and title not in titulos_vistos:
                    titulos_vistos.add(title)
                    fout.write(json.dumps({"title": title, "content": content}, ensure_ascii=False) + "\n")
                    linhas_escritas += 1
            except (json.JSONDecodeError, AttributeError):
                pass  # Ignora linhas malformadas

    print(f"Limpeza concluída. Lidas: {linhas_lidas}, Válidas e Únicas: {linhas_escritas}")
    return linhas_escritas


def carregar_registros_jsonl(caminho:str):
    """Carrega registros de um arquivo JSONL para uma lista de dicionários."""
    registros = []
    with open(caminho, "r", encoding="utf-8") as f:
        for i, linha in enumerate(f, 1):
            linha = linha.strip()
            if not linha:
                continue
            try:
                obj = json.loads(linha)
                title = html.unescape(obj.get("title","").strip())
                content = html.unescape(obj.get("content","").strip())
                if title and content:
                    registros.append({"title": title, "content": content})
            except json.JSONDecodeError:
                pass
    return registros

def converter_para_formato_instruction(registros, instrucao="DESCRIBE THIS PRODUCT"):
    """Converte uma lista de registros para o formato de instrução (instruction/input/output)."""
    return {
        "instruction": [instrucao]*len(registros),
        "input": [r["title"] for r in registros],
        "output": [r["content"] for r in registros],
    }

def salvar_json(dados, caminho:str):
    """Salva um dicionário como um arquivo JSON formatado."""
    with open(caminho, "w", encoding="utf-8") as f:
        json.dump(dados, f, ensure_ascii=False, indent=2)
    print(f"Arquivo salvo: {caminho}")

# Gera data_titles_contents_cleaned.jsonl a partir do arquivo bruto
if not Path(DATA_RAW_PATH).exists():
    raise FileNotFoundError(f"Arquivo bruto não encontrado: {DATA_RAW_PATH}")

print("Limpeza e criação do JSONL...")
_ = limpar_arquivo_raw_para_jsonl(DATA_RAW_PATH, CLEAN_JSONL_PATH)

# Amostra de linhas limpas
with open(CLEAN_JSONL_PATH, "r", encoding="utf-8") as f:
    for i in range(3):
        print(f"Exemplo linha limpa {i+1}: {f.readline().strip()}")

registros = carregar_registros_jsonl(CLEAN_JSONL_PATH)
print(f"Total de registros carregados: {len(registros)}")

dataset_chat = converter_para_formato_instruction(registros, "DESCRIBE THIS PRODUCT")
salvar_json(dataset_chat, FORMAT_DATA_JSON)

print("Amostra:")
for i in range(2):
    print(f"Instrução: {dataset_chat['instruction'][i]}")
    print(f"Input: {dataset_chat['input'][i]}")
    print(f"Output(len={len(dataset_chat['output'][i])}): {dataset_chat['output'][i][:120]}...")
    print("-"*60)

# Carrega o JSON (listas)
hf_ds = Dataset.from_dict(dataset_chat)

# Adiciona um índice para referência
hf_ds = hf_ds.add_column("idx", list(range(len(hf_ds))))

# Split treino/val (ex: 95% treino / 5% validação)
perc_valid = 0.05
n_valid = max(1, int(len(hf_ds)*perc_valid))
hf_ds = hf_ds.shuffle(seed=SEED)
valid_ds = hf_ds.select(range(n_valid))
train_ds = hf_ds.select(range(n_valid, len(hf_ds)))

print(f"Tamanho treino: {len(train_ds)} | validação: {len(valid_ds)}")

dataset_dict = DatasetDict({
    "train": train_ds,
    "validation": valid_ds
})

print(dataset_dict)


In [ ]:

### 3. Formatação do Prompt (Estilo Alpaca)
#
# Para que o modelo entenda a tarefa, formatamos cada exemplo de treino e validação usando um *template* de prompt. Escolhemos o formato "Alpaca", que é amplamente compatível com modelos de instrução.
#
# O template estrutura a informação da seguinte forma:
# -   **Instruction**: A tarefa que o modelo deve executar (ex: "DESCRIBE THIS PRODUCT").
# -   **Input**: O contexto específico para a tarefa (o título do produto).
# -   **Response**: Onde o modelo deve gerar sua resposta (a descrição do produto).
#
# Essa formatação é crucial para o sucesso do fine-tuning.

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(example_batch):
    instructions = example_batch["instruction"]
    inputs = example_batch["input"]
    outputs = example_batch["output"]
    texts = []
    for inst, inp, outp in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(inst, inp, outp)
        texts.append(text)
    return {"text": texts}

formatted_train = dataset_dict["train"].map(formatting_prompts_func, batched=True, num_proc=1)
formatted_valid = dataset_dict["validation"].map(formatting_prompts_func, batched=True, num_proc=1)

print("Exemplo texto formatado:\n")
print(formatted_train[0]["text"][:500])


In [ ]:

### 4. Carregamento do Modelo Base e Avaliação de Baseline
#
# Antes de treinar, precisamos de um ponto de partida (baseline) para medir o progresso.
#
# 1.  **Carregamento do Modelo**: Usamos `unsloth/llama-3-8b-bnb-4bit`, uma versão otimizada do Llama 3 8B que consome menos memória graças à quantização em 4-bit. A biblioteca `Unsloth` acelera significativamente o carregamento e o treinamento.
# 2.  **Geração de Baseline**: Pegamos algumas amostras do conjunto de validação e pedimos ao modelo (ainda não treinado) para gerar descrições. Isso nos mostra como o modelo se comporta "de fábrica".
# 3.  **Análise**: Como esperado, as descrições geradas pelo modelo base são genéricas, repetitivas ou simplesmente erradas, pois ele ainda não foi especializado na nossa tarefa. Salvamos esses resultados em `baseline_samples.csv` para comparação posterior.

model_name = "unsloth/llama-3-8b-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = DTYPE,
    load_in_4bit = LOAD_IN_4BIT,
)

print("Modelo base carregado.")

def gerar(model, tokenizer, instruction, input_text, max_new_tokens=128):
    prompt = alpaca_prompt.format(instruction, input_text, "")
    inputs = tokenizer([prompt], return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=max_new_tokens, use_cache=True)
    decoded = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    # Extrair somente a parte depois de ### Response: (heurística simples)
    if "### Response:" in decoded:
        decoded = decoded.split("### Response:")[1].strip()
    return decoded

SAMPLES_BASELINE = min(5, len(formatted_valid))
baseline_records = []
for i in range(SAMPLES_BASELINE):
    row = formatted_valid[i]
    instruction = row["instruction"]
    input_title = row["input"]
    ref_output = row["output"]
    gen = gerar(model, tokenizer, instruction, input_title)
    baseline_records.append({
        "idx": row["idx"],
        "title": input_title,
        "ref": ref_output,
        "gen_before": gen
    })

baseline_df = pd.DataFrame(baseline_records)
baseline_path = os.path.join(RESULTS_DIR, "baseline_samples.csv")
baseline_df.to_csv(baseline_path, index=False)
print(baseline_df)


In [ ]:

### 5. Configuração do Fine-Tuning com LoRA
#
# Em vez de treinar o modelo inteiro (o que seria computacionalmente caro), usamos a técnica **LoRA (Low-Rank Adaptation)**.
#
# -   **Como funciona?**: O LoRA congela os pesos originais do modelo e treina apenas pequenos "adaptadores" que são inseridos em camadas específicas (geralmente as de atenção).
# -   **Vantagens**:
#     -   **Eficiência**: Reduz drasticamente o número de parâmetros treináveis (de bilhões para alguns milhões).
#     -   **Velocidade**: O treinamento é muito mais rápido.
#     -   **Portabilidade**: Os adaptadores treinados são pequenos (alguns megabytes), facilitando o armazenamento e o compartilhamento.
#
# Configuramos o LoRA para ser aplicado às principais camadas de projeção do modelo (`q_proj`, `k_proj`, `v_proj`, etc.), que são cruciais para o aprendizado.

model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Aumentado de 16 para 32
    lora_alpha = 64, # Aumentado de 16 para 64 (dobro de r)
    lora_dropout = 0.1, # Adicionado dropout de 10%
    target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = SEED,
    use_rslora = False,
    loftq_config = None,
)

print("Modelo adaptado para LoRA.")


In [ ]:

### 6. Treinamento do Modelo
#
# Com tudo configurado, iniciamos o treinamento usando o `SFTTrainer` (Supervised Fine-tuning Trainer) da biblioteca `trl`.
#
# -   **`TrainingArguments`**: Definimos os hiperparâmetros do treinamento, como:
#     -   `per_device_train_batch_size` e `gradient_accumulation_steps`: Controlam o tamanho efetivo do batch, otimizando o uso de memória.
#     -   `learning_rate`: A taxa de aprendizado.
#     -   `max_steps`: O número total de passos de treinamento.
#     -   `logging_steps`: A frequência com que o progresso do treino é registrado.
# -   **`SFTTrainer`**: Orquestra todo o processo, alimentando o modelo com os dados formatados e aplicando as otimizações do LoRA e do Unsloth.
#
# Ao final, os adaptadores LoRA treinados são salvos no diretório `lora_adapters`.

training_args = TrainingArguments(
    output_dir = "outputs",
     num_train_epochs = EPOCHS,
    per_device_train_batch_size = BATCH_SIZE,
    gradient_accumulation_steps = GRAD_ACCUM,
    warmup_steps = WARMUP_STEPS,
    max_steps = MAX_STEPS,
    learning_rate = LR,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    logging_steps = LOGGING_STEPS,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = SEED,
    # evaluation_strategy = "no",  # pode alterar para "steps" se quiser avaliar durante treino
    save_strategy = "steps",
    save_steps = 200,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train,
    dataset_text_field = "text",
    max_seq_length = MAX_SEQ_LENGTH,
    dataset_num_proc = 1,
    packing = False,
    args = training_args,
)

print("Trainer configurado.")

train_result = trainer.train()
trainer.model.save_pretrained("lora_adapters")
tokenizer.save_pretrained("lora_adapters")

with open(os.path.join(RESULTS_DIR, "training_log.txt"), "w") as f:
    f.write(str(train_result))

print("Treinamento concluído.")

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "DESCRIBE THIS PRODUCT",
        "A Day in the Life of China", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
print(tokenizer.batch_decode(outputs))


In [ ]:

### 7. Avaliação Pós-Treinamento e Comparação
#
# Agora, a parte mais importante: **o modelo melhorou?**
#
# 1.  **Preparação para Inferência**: Carregamos os adaptadores LoRA no modelo base. O Unsloth otimiza esse processo com `FastLanguageModel.for_inference()`.
# 2.  **Geração em Batch**: Para avaliar o desempenho no conjunto de validação de forma eficiente, usamos um `pipeline` da `transformers` para gerar todas as descrições em *batch* (lotes). Isso é muito mais rápido do que gerar uma por uma.
# 3.  **Coleta de Resultados**: Armazenamos as descrições geradas pelo modelo *fine-tuned* em `validation_generation_after_batch.csv`.
# 4.  **Comparação Lado a Lado**: Criamos um DataFrame que mostra, para cada produto:
#     -   O título (`title`).
#     -   A descrição original (`ref`).
#     -   A descrição gerada *antes* do treino (`gen_before`).
#     -   A descrição gerada *depois* do treino (`gen_after`).
#
# Isso nos permite visualizar qualitativamente a melhoria. As respostas pós-treino devem ser muito mais precisas e contextuais.

import pandas as pd
import os
from transformers import pipeline
from tqdm import tqdm # Ótimo para visualizar o progresso

# Supondo que 'model' e 'tokenizer' já foram carregados e otimizados com Unsloth
# FastLanguageModel.for_inference(model) # Você já fez isso, ótimo!

# 1. Crie um pipeline de geração de texto
#    - device=0 significa usar a primeira GPU.
#    - O pipeline cuidará do batching automaticamente.
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # device=0 # Coloque o pipeline na GPU - REMOVIDO DEVIDO AO USO DE accelerate
)

# 2. Prepare todas as suas instruções e entradas (prompts) em uma lista
#    Esta é a formatação que o modelo espera. Adapte se a sua for diferente.
#    Exemplo de formatação para Alpaca/Llama:
prompts = []
for row in formatted_valid:
    instruction = row["instruction"]
    input_title = row["input"]

    # Adapte este template de prompt para o que seu modelo foi treinado
    prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_title}

### Response:
"""
    prompts.append(prompt)

# 3. Execute a geração em batch
#    - Defina um `batch_size` que caiba na sua memória. Comece com 8 ou 16 e ajuste.
#    - `max_new_tokens` define o comprimento máximo da resposta gerada.
#    - `pad_token_id` é importante para o batching funcionar corretamente.
print("Iniciando a geração em batch...")
generated_outputs = text_generator(
    prompts,
    max_new_tokens=256,  # Ajuste conforme necessário
    batch_size=8,        # <<-- PONTO CRÍTICO: Ajuste este valor!
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,      # Configurações de geração
    top_p=0.9,
    temperature=0.7,
)
print("Geração concluída.")

# 4. Processe os resultados
eval_records = []
for i, row in enumerate(formatted_valid):
    # O pipeline retorna uma lista de listas de dicionários.
    # Pegamos o texto gerado do primeiro (e único) resultado para cada prompt.
    gen_after = generated_outputs[i][0]['generated_text']

    # O resultado inclui o prompt. Vamos remover o prompt para ter apenas a resposta.
    # Isso garante que 'gen_after' contenha apenas o texto novo.
    gen_only = gen_after[len(prompts[i]):].strip()

    eval_records.append({
        "idx": row["idx"],
        "title": row["input"],
        "ref": row["output"],
        "gen_after": gen_only
    })

# 5. Salve os resultados como antes
eval_df = pd.DataFrame(eval_records)
eval_path = os.path.join(RESULTS_DIR, "validation_generation_after_batch.csv")
eval_df.to_csv(eval_path, index=False)
print(eval_df.head())


In [ ]:
!pip install rouge_score

In [ ]:

### 8. Métricas de Avaliação Quantitativa
#
# Para medir a melhoria de forma objetiva, calculamos várias métricas padrão:
#
# -   **ROUGE (Recall-Oriented Understudy for Gisting Evaluation)**: Mede a sobreposição de n-grams (sequências de palavras) entre o texto gerado e a referência. É bom para avaliar a qualidade do conteúdo.
# -   **BLEU (Bilingual Evaluation Understudy)**: Mede a precisão, focando em quão similar é o texto gerado em relação à referência. É mais rígido que o ROUGE.
# -   **BERTScore**: Em vez de olhar apenas para palavras, usa embeddings BERT para medir a similaridade semântica. É uma métrica mais robusta, pois entende o significado das palavras.
# -   **Token Overlap**: Uma métrica simples que calcula a sobreposição de tokens (palavras únicas) entre o gerado e a referência.
#
# Comparamos os scores *antes* e *depois* do fine-tuning. Esperamos ver um aumento significativo em todas as métricas, confirmando que o modelo aprendeu a gerar descrições mais relevantes. Os resultados são salvos em `evaluation_metrics.json`.

comparacao_df = pd.merge(baseline_df, eval_df[["idx","gen_after"]], on="idx", how="left")
comparacao_path = os.path.join(RESULTS_DIR, "comparacao_baseline_after.csv")
comparacao_df.to_csv(comparacao_path, index=False)
print(comparacao_df)

# CÁLCULO DE MÉTRICAS DE AVALIAÇÃO
rouge_metric = load_metric("rouge")
sacrebleu_metric = load_metric("sacrebleu")

def calculate_metrics(df):
    """Calcula ROUGE, BLEU, BERTScore e Overlap de Tokens."""

    # Filtra linhas onde a geração pós-treino não é nula
    df_valid = df.dropna(subset=['gen_after'])

    # Extrai referências e predições
    references = df_valid["ref"].tolist()
    predictions_before = df_valid["gen_before"].tolist()
    predictions_after = df_valid["gen_after"].tolist()

    # Garante que as predições não sejam listas vazias
    if not predictions_after:
        print("Nenhuma predição válida encontrada para calcular métricas.")
        return {}

    # --- ROUGE ---
    rouge_before = rouge_metric.compute(predictions=predictions_before, references=references)
    rouge_after = rouge_metric.compute(predictions=predictions_after, references=references)

    # --- BLEU ---
    # Sacrebleu espera uma lista de listas de referências
    bleu_references = [[ref] for ref in references]
    bleu_before = sacrebleu_metric.compute(predictions=predictions_before, references=bleu_references)
    bleu_after = sacrebleu_metric.compute(predictions=predictions_after, references=bleu_references)

    # --- BERTScore ---
    # Calcula BERTScore (P, R, F1)
    P_before, R_before, F1_before = bert_scorer(predictions_before, references, lang="en", model_type="bert-base-uncased", device=DEVICE)
    P_after, R_after, F1_after = bert_scorer(predictions_after, references, lang="en", model_type="bert-base-uncased", device=DEVICE)

    bertscore_before = {"precision": P_before.mean().item(), "recall": R_before.mean().item(), "f1": F1_before.mean().item()}
    bertscore_after = {"precision": P_after.mean().item(), "recall": R_after.mean().item(), "f1": F1_after.mean().item()}

    # --- Token Overlap ---
    def token_overlap(text1, text2):
        tokens1 = set(text1.lower().split())
        tokens2 = set(text2.lower().split())
        intersection = len(tokens1.intersection(tokens2))
        union = len(tokens1.union(tokens2))
        return intersection / union if union > 0 else 0

    overlap_before = np.mean([token_overlap(p, r) for p, r in zip(predictions_before, references)])
    overlap_after = np.mean([token_overlap(p, r) for p, r in zip(predictions_after, references)])

    metrics = {
        "before_finetune": {
            "rouge": rouge_before,
            "bleu": bleu_before,
            "bert_score": bertscore_before,
            "token_overlap": overlap_before
        },
        "after_finetune": {
            "rouge": rouge_after,
            "bleu": bleu_after,
            "bert_score": bertscore_after,
            "token_overlap": overlap_after
        }
    }

    return metrics

# Calcular e exibir as métricas
evaluation_metrics = calculate_metrics(comparacao_df)

# Salvar métricas em um arquivo JSON
metrics_path = os.path.join(RESULTS_DIR, "evaluation_metrics.json")
with open(metrics_path, "w") as f:
    json.dump(evaluation_metrics, f, indent=4)

print("Métricas de avaliação calculadas e salvas.")
print(json.dumps(evaluation_metrics, indent=2))


In [ ]:

### 9. Simulação de Caso de Uso: Busca por Produto e Geração de Descrição
#
# Esta seção simula a aplicação prática do nosso sistema, conforme solicitado no desafio.
#
# 1.  **Consulta do Usuário**: Recebemos uma pergunta em linguagem natural sobre um produto (ex: "tell me about the ballet tutu for girls").
# 2.  **Busca por Similaridade (Fuzzy Matching)**: Usamos a biblioteca `rapidfuzz` para encontrar o título de produto mais similar à consulta do usuário em nossa lista de produtos. Isso lida com erros de digitação, sinônimos ou formulações diferentes.
# 3.  **Geração de Resposta**: Uma vez que o produto é identificado, passamos seu título exato para o modelo *fine-tuned*, que gera a descrição detalhada e precisa que aprendeu durante o treinamento.
#
# Este fluxo de trabalho demonstra um sistema de "busca e resposta" ponta a ponta, onde o usuário pode encontrar informações sobre um produto sem precisar saber seu nome exato.

def generate_response(instruction, input_text):
    # Format the input using the alpaca prompt
    prompt = alpaca_prompt.format(instruction, input_text, "")

    # Tokenize the input
    inputs = tokenizer(
        [prompt],
        return_tensors="pt"
    ).to("cuda")

    # Generate the response
    outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)

    # Decode and return the response
    response = tokenizer.batch_decode(outputs)[0]

    # Extrai somente a parte depois de ### Response: (heurística simples)
    response_start = response.find("### Response:\n") + len("### Response:\n")
    return response[response_start:].replace(tokenizer.eos_token, "").strip()

# Example usage:
instruction = "DESCRIBE THIS PRODUCT"
input_text = "Girls Ballet Tutu Neon Pink"

response = generate_response(instruction, input_text)
print(response)

# CARREGAR TÍTULOS PARA BUSCA
# Garante que temos a lista de todos os títulos dos produtos para a busca por similaridade.
all_titles = [r["title"] for r in registros]
print(f"Total de {len(all_titles)} títulos carregados para a busca por similaridade.")

def find_and_describe_product(user_query: str, titles_list: list, similarity_threshold=80):
    """
    Encontra o título mais similar a uma consulta de usuário e gera a descrição do produto.

    1. Usa fuzzy matching para encontrar o título mais próximo na lista de produtos.
    2. Se a similaridade for alta o suficiente, usa o modelo para gerar a descrição.
    3. Retorna o título encontrado e a descrição gerada.
    """
    # Encontra o melhor match para a consulta do usuário
    # `process.extractOne` retorna uma tupla: (título, score, índice)
    best_match = process.extractOne(user_query, titles_list, scorer=fuzz.WRatio)

    if not best_match or best_match[1] < similarity_threshold:
        return "Produto não encontrado.", f"Nenhum produto com similaridade acima de {similarity_threshold}% encontrado para '{user_query}'.", None

    found_title, score, _ = best_match
    print(f"Consulta do usuário: '{user_query}'")
    print(f"Título mais similar encontrado: '{found_title}' (Similaridade: {score:.2f}%)")

    # Gera a descrição para o título encontrado usando o modelo fine-tuned
    print("\nGerando descrição com o modelo...")
    generated_description = generate_response("DESCRIBE THIS PRODUCT", found_title)

    return found_title, generated_description, score

# --- Exemplo de uso da função de busca e geração ---
user_question = "Can you tell me about the ballet tutu for girls?"
found_title, description, score = find_and_describe_product(user_question, all_titles)

print("\n--- RESULTADO FINAL ---")
print(f"Produto Correspondente: {found_title}")
print(f"Descrição Gerada:\n{description}")


In [ ]:

### 10. Salvando o Modelo para Produção (Opcional)
#
# Após o treinamento e a validação, o próximo passo seria preparar o modelo para ser usado em um ambiente de produção.
#
# -   **Mesclar Adaptadores**: Para simplificar a implantação, podemos mesclar os pesos do LoRA diretamente no modelo base. Isso cria um novo modelo que não precisa mais dos arquivos de adaptadores separados. O Unsloth facilita isso com `model.merge_and_unload()`.
# -   **Salvar Modelo Mesclado**: O modelo completo e mesclado pode ser salvo usando `model.save_pretrained("merged_model")`.
# -   **Quantização GGUF**: Para máxima portabilidade e eficiência (especialmente para rodar em CPUs ou dispositivos com menos VRAM), podemos converter o modelo para o formato **GGUF**. Isso o torna compatível com ferramentas como `llama.cpp`.
#
# Esta etapa garante que o modelo possa ser facilmente integrado a aplicações como APIs, chatbots ou sistemas de busca.

# (Opcional) Mesclar adaptadores e salvar modelo pronto para inferência standalone
# Esta etapa é útil para criar um modelo único que não depende mais dos adaptadores LoRA.

# 1. Mesclar os pesos do LoRA com o modelo base
# O modelo agora se comporta como um modelo padrão da Hugging Face
# model = model.merge_and_unload() # Descomente para mesclar

# 2. Salvar o modelo mesclado (formato Hugging Face)
# model.save_pretrained("llama3-8b-product-desc-merged")
# tokenizer.save_pretrained("llama3-8b-product-desc-merged")

# 3. (Avançado) Salvar no formato GGUF para inferência em CPU com llama.cpp
# model.save_pretrained_gguf("llama3-8b-product-desc", tokenizer, quantization_method="q4_k_m")

print("Adaptadores salvos em ./lora_adapters.")
print("Para salvar o modelo mesclado ou em formato GGUF, descomente as linhas nesta célula.")

## Resumo Final
#
# Artefatos gerados:
# - data_titles_contents_cleaned.jsonl (dados limpos)
# - formatted_products_chat_data.json (instruction/input/output)
# - baseline_samples.csv (amostras antes do treino)
# - validation_generation_after.csv (gera pós-treino)
# - comparacao_baseline_after.csv (comparativo)
# - metrics.json (ROUGE, BLEU, overlap)
# - lora_adapters/ (pesos LoRA)
# - batch_queries_results.json (exemplos de perguntas)
# - outputs/ (logs do Trainer)
#
# Próximos Passos Recomendados:
# 1. Aumentar número de passos / épocas.
# 2. Aplicar filtragem de descrições muito curtas/longas.
# 3. Introduzir truncamento inteligente (tokenizer truncation).
# 4. Adicionar métricas de similaridade semântica (BERTScore).
# 5. Publicar adaptadores no HuggingFace Hub.
# 6. Integrar a um endpoint (FastAPI / Gradio).
#
# Fim.